In [38]:
import pandas as pd
import numpy as np
from sklearn.neighbors import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.compose import *

import matplotlib.pyplot as plt

In [39]:
df = pd.read_csv(r"/home/sarthakredasani/Documents/CDAC_ML/Cases/Cases/Medical Cost Personal/insurance.csv")
df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [40]:
X, y = df.drop('charges', axis =1), df['charges']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=25)

In [41]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
ct = make_column_transformer((ohe, make_column_selector(dtype_include=object)),
                             ('passthrough', make_column_selector(dtype_exclude=object)), 
                             verbose_feature_names_out=False)
ct = ct.set_output(transform='pandas')

In [42]:
X_trn_ohe = ct.fit_transform(X_train)
X_tst_ohe = ct.transform(X_test)

In [43]:
ks = np.arange(1,21)
scores = []
for i in ks:
    knn = KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_trn_ohe, y_train)
    y_pred = knn.predict(X_tst_ohe)
    scores.append([i , r2_score(y_test, y_pred)])
df_scores = pd.DataFrame(scores, columns=['k', 'score'])
df_scores.sort_values('score', ascending=False).iloc[:3]

,k,score
10,11,0.167058
6,7,0.165495
9,10,0.164197


In [44]:
# Standard Scaling

In [45]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
scaler = StandardScaler().set_output(transform='pandas')
ct = make_column_transformer((ohe, make_column_selector(dtype_include=object)), 
                             (scaler,make_column_selector(dtype_exclude=object)),
                              verbose_feature_names_out=False)
ct = ct.set_output(transform='pandas')

In [46]:
X_trn_ss = ct.fit_transform(X_train)
X_tst_ss = ct.transform(X_test)

In [47]:
Ks = np.arange(1,21)
scores = []
for i in Ks:
    knn = KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_trn_ss, y_train)
    y_pred = knn.predict(X_tst_ss)
    scores.append([i, r2_score(y_test, y_pred)])
df_scores = pd.DataFrame(scores, columns=['K', 'score'])
df_scores.sort_values('score', ascending=False).iloc[:3]

,K,score
4,5,0.658890
2,3,0.656746
3,4,0.655613


In [48]:
# Min Max Scaler

In [49]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
scaler = MinMaxScaler().set_output(transform='pandas')
ct = make_column_transformer((ohe, make_column_selector(dtype_include=object)), 
                             (scaler,make_column_selector(dtype_exclude=object)),
                              verbose_feature_names_out=False)
ct = ct.set_output(transform='pandas')

In [50]:
X_trn_mm = ct.fit_transform(X_train)
X_tst_mm = ct.transform(X_test)

In [51]:
Ks = np.arange(1,21)
scores = []
for i in Ks:
    knn = KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_trn_mm, y_train)
    y_pred = knn.predict(X_tst_mm)
    scores.append([i, r2_score(y_test, y_pred)])
df_scores = pd.DataFrame(scores, columns=['K', 'score'])
df_scores.sort_values('score', ascending=False).iloc[:3]

,K,score
4,5,0.746992
5,6,0.745948
3,4,0.743858


In [52]:
# Inferencing

In [53]:
tst = pd.read_csv(r"/home/sarthakredasani/Documents/CDAC_ML/Cases/Cases/Medical Cost Personal/tst_insure.csv")
tst.head(5)

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,37,female,23.370,2,no,northwest


In [54]:
from sklearn.pipeline import Pipeline


scl_std = StandardScaler()
ohe = OneHotEncoder(sparse_output=False, drop='first')
ct = make_column_transformer((ohe, make_column_selector(dtype_include=object)), 
                             ( "passthrough" ,make_column_selector(dtype_exclude=object)),
                              verbose_feature_names_out=False)
knn = KNeighborsRegressor(n_neighbors=5)
best_model = Pipeline([('OHE', ct), ('SCL', scl_std), ('KNN', knn)])

In [55]:
best_model.fit(X,y)

Pipeline(steps=[('OHE',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd7c189ee10>),
                                                 ('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd7c189c510>)],
                                   verbose_feature_names_out=False)),
                ('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())])

In [56]:
best_model.predict(tst)

array([27374.3606   ,  6923.43421  ,  4923.2172   ,  7746.225252 ,
        6711.63442  ,  7749.9086   ,  1843.61314  ,  2437.65453  ,
       36819.43778  ,  9866.67163  ,  3304.91078  , 27374.3606   ,
        9362.9852   , 22295.73195  ,  6446.1202   , 12201.35051  ,
       13550.3241   , 16329.129206 ,  1859.79761  ,  7106.0190738,
        2054.14964  ,  5739.269666 ,  2037.28478  , 17155.436028 ,
        3722.421466 , 14104.432552 ,  6860.167622 , 22792.585194 ,
        6888.03737  , 30894.566528 ,  4835.75698  , 40915.031192 ,
       12574.459432 , 13896.467678 ,  1963.57     ,  4883.13423  ,
       14769.495438 , 11208.819282 , 19548.61324  ,  9250.472868 ,
       12304.673362 , 21226.36544  , 19840.152966 , 37154.79512  ,
       17155.436028 , 24952.13688  , 37386.98846  , 10770.2092   ,
        8390.300026 ,  4203.17097  ,  9575.48302  ,  5184.436    ,
        8311.08301  , 10889.58527  , 16652.336602 ,  2302.20296  ,
        8844.40044  , 15894.916494 , 37386.98846  ,  8240.314 